
1) размечалка + манифест +

2) признаки на основе предыдущих (следующих) строк +

3) признак - средний размер баундин бокса +

4) gboost +, xgboost +

5) жирность + (средний цвет ббокса) + средний цвет ббоксов +

6) кросс-валидация по группам +

7) посмотреть, где ошибается классификатор

8) количество букв в строке (+ среднее количество букв по докам) +

9) количество букв первого слова +

10) тюнить параметры

In [1]:
import re
import cv2
import json
import numpy as np
from functools import cmp_to_key
from functools import reduce
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, cohen_kappa_score
from xgboost import XGBClassifier
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
reg_expr = [re.compile(r'\d+(\.\d+)+\D'), # 1.1.1
            re.compile(r'\d\)'), # 1)
            re.compile(r'\w(\.\w)+\W'), # b.b.b
            re.compile(r'[а-яА-Я\d](\.[а-яА-Я\d])+\W'), # б.б.б
            re.compile(r'[a-zA-Z]\)'), # a)
            re.compile(r'[a-zA-Z]\.\W'), # b.
            re.compile(r'[а-яА-Я]\)'), # б)
            re.compile(r'[а-яА-Я]\.\W'), # б.
            re.compile(r'\-'), # -
            re.compile(r'\*'), # *
            re.compile(r'Раздел|Подраздел|Глава|Параграф|Секция|Часть|Статья')]

def add_reg_features(line):
    features = [0] * len(reg_expr)
    i = 0
    for expr in reg_expr:
        match = expr.search(line)
    
        if match:
            if match.start() == 0:
                features[i] = 1
                return features
        i += 1
        
    return features

In [3]:
def mean_bbox_size(bboxes):
    """
    bboxes - list [{"text": "", "bbox": []}, {} ...]
    returns (mean_height, mean_width)
    """
    lefts = np.sum(list(map(lambda x: x['bbox'][0], bboxes)))
    widths = np.sum(list(map(lambda x: x['bbox'][2], bboxes)))
    heigths = np.sum(list(map(lambda x: x['bbox'][3], bboxes)))
    num_bboxes = len(bboxes)
    
    return (lefts / num_bboxes, heigths / num_bboxes, widths / num_bboxes)

In [4]:
# add simple font-weight detection

def mean_color(bbox, img):
    x, y, w, h = bbox
    crop_img = img[y:y + h, x:x + w]
    avg_color_per_row = np.average(crop_img, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    return np.average(avg_color)

In [5]:
# amount letters of the first word in line

RE_WORD = re.compile(r'[a-zA-Zа-яА-Я]+')

def letters_cnt(line):
    match = RE_WORD.search(line)

    if match:
        if match.start() == 0: # word in the beginning of the line
            return len(match.group(0))
        else:
            return 0

In [18]:
class AddImgFeatures:
    def __init__(self):
        pass
    
    def fit(self):
        pass
    
    def transform(self, X):
        """
        X - [{"name": "doc_name", "entities": [{"text": "", "bbox": []}]}]
        returns features [normalized bbox sizes, normalized mean size of bbox]
        [normalized left, normalized top,
        normalized width, normalized height,
        mean height, mean width]
        """
        features = []
        for doc in X:
            doc_features = []
            doc_name = doc['name']
            doc_info = doc['entities']
            
            sum_avg_colors = 0
            sum_line_cnt = 0
            
            mean_left, mean_heigth, mean_width = mean_bbox_size(doc_info)
            img = cv2.imread('docs/' + doc_name)
            
            #avg_color_per_row = np.average(img, axis=0)
            #avg_color = np.average((np.average(avg_color_per_row, axis=0)))
            
            heigth = img.shape[0]
            width = img.shape[1]
            for line_info in doc_info:
                line_color = mean_color(line_info['bbox'], img)
                line_features = [line_info['bbox'][0] / width,
                                line_info['bbox'][1] / heigth,
                                line_info['bbox'][2] / width,
                                line_info['bbox'][3] / heigth,
                                mean_left, mean_heigth, mean_width,
                                line_color]
                sum_avg_colors += line_color
                line_features += add_reg_features(line_info['text'])
                
                word_cnt = letters_cnt(line_info['text'])
                line_cnt = len(line_info['text'])
                line_features += [word_cnt, line_cnt]
                sum_line_cnt += line_cnt
                
                doc_features.append(line_features)
            mean_line_color = sum_avg_colors / len(doc_info)
            mean_line_cnt = sum_line_cnt / len(doc_info)
            for line_features in doc_features:
                line_features += [mean_line_cnt, mean_line_color]
            features.append(doc_features)
        return features
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [8]:
def add_prev_next_features(doc, line_features, num_line):
    """
    doc - list of line_features
    line_features - list of features
    """
    add_f = [0] * len(line_features)
    extended_doc = [add_f] * 4 + doc + [add_f] * 4
    prev_features = reduce(lambda x, y: x + y, 
                           extended_doc[num_line: num_line + 4])
    next_features = reduce(lambda x, y: x + y, 
                           extended_doc[num_line + 5: num_line + 9])
    return line_features + prev_features + next_features

In [9]:
class AddPrevNextFeatures:
    def __init__(self):
        pass
    
    def fit(self):
        pass
    
    def transform(self, X):
        """
        X - list of doc_features
        doc_features - list of line_features
        
        for each line 4 previous and 4 next features added
        result list of lines features
        """
        result = []
        for doc_features in X:
            i = 0
            for line_features in doc_features:
                new_features = add_prev_next_features(doc_features, line_features, i)
                i += 1
                result.append(new_features)
        return np.array(result)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

https://xgboost.readthedocs.io/en/latest/tutorials/index.html

In [19]:
label2class = {
    "header" : 1,
    "list" : 2,
    "text" : 3,
    "other" : 4
}

with open("data.json", "r") as read_file:
    docs = json.load(read_file)
    y = []
    for doc in docs:
        elem = [label2class[line['label']] for line in doc['entities']]
        y.append(elem)
    X = []
    for doc in docs:
        elem = {}
        elem['name'] = doc['name']
        elem['entities'] = [{'text': line_info['text'], 
                        'bbox': [line_info['x'], line_info['y'],
                                line_info['width'], line_info['height']]}
                      for line_info in doc['entities']]
        X.append(elem)
    X = np.array(X)
    y = np.array(y)

https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

parameters tuning:

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [20]:
clf = make_pipeline(AddImgFeatures(),
                    AddPrevNextFeatures(),
                    XGBClassifier())

scores = []

n_folds = 3
kf = KFold(n_splits=n_folds)
for train_index, test_index in tqdm(kf.split(X), total=n_folds):
    X_train, X_test = X[train_index], X[test_index]
    y_train = np.array(reduce(lambda x1, x2: x1 + x2, y[train_index]))
    y_test = np.array(reduce(lambda x1, x2: x1 + x2, y[test_index]))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    scores.append(f1_score(y_test, y_pred, average='macro'))

print(scores)
print(np.mean(scores))

/Users/anastasiabogatenkova/miniconda3/envs/doc-py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



[0.885195704419441, 0.8303121752365356, 0.8352649206685879]
0.8502576001081882


XGBoost 3 folds

[0.8398772119663552, 0.8103215605086755, 0.8062687365220652]
0.818822502999032

num_estimators = 500:

[0.8561968437477718, 0.8142261914976299, 0.8242253530026652]
0.8315494627493556

+ len of each line:

[0.8075060113332115, 0.7928847932896882, 0.8238502119065526]
0.8080803388431508

num_estimators = 100

+ len of each line:

[0.8027224789705375, 0.8112443626161243, 0.8253655342327253]
0.8131107919397956

+ len of first word:

[0.8601182793773471, 0.8337325172195696, 0.8447938565935867]
0.8462148843968346

+ mean len of each line:

[0.8870400621195444, 0.833668982848783, 0.8341967455729354]
0.8516352635137543

+ mean color of each line in document:

[0.8854012889597398, 0.8414174427949209, 0.8465237398017204]
0.857780823852127

+ delete avg_color feature

[0.885195704419441, 0.8303121752365356, 0.8352649206685879]
0.8502576001081882

feature importances https://github.com/limexp/xgbfir

In [13]:
clf = make_pipeline(AddImgFeatures(),
                    AddPrevNextFeatures(),
                    XGBClassifier())

X_train, X_test = X[:400], X[400:]
y_train = np.array(reduce(lambda x1, x2: x1 + x2, y[:400]))
y_test = np.array(reduce(lambda x1, x2: x1 + x2, y[400:]))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
score = f1_score(y_test, y_pred, average='macro')
score

0.8642870085554297

X_test [{"name": "doc_name", "entities": [{"text": "", "bbox": []}]}]

In [14]:
d = {}
i = 0
for doc_info in X_test:
    for line_info in doc_info['entities']:
        d[(tuple(line_info['bbox']), doc_info['name'], line_info["text"])] = [y_test[i], y_pred[i]]
        i += 1

In [15]:
color_dict = {
    1 : (0, 0, 255),
    2 : (0, 255, 0),
    3 : (255, 0, 0),
    4 : (0, 255, 255)
}
class2label = {
    1: "header", 
    2: "list", 
    3: "text", 
    4: "other"
}

grouped_by_dock = defaultdict(list)
for item in d.items():
    key, (real_class, predicted_class) = item
    file_name = key[1]
    grouped_by_dock[file_name].append(item)


for file_name, items in tqdm(grouped_by_dock.items()):
    
    img = None 
    for key, (real_class, predicted_class) in items:        
        if real_class != predicted_class:   
            if img is None:
                img = cv2.imread('docs/' + file_name)
            (x, y, w, h) = key[0]
            cv2.rectangle(img, (x, y), (x + w, y + h), color_dict[predicted_class], 2)
            cv2.putText(img, class2label[predicted_class] + " " + class2label[real_class], 
                        (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, 
                        color_dict[predicted_class], 2)
    if img is not None:
        cv2.imwrite('different_docs/' + file_name, img)        

/Users/anastasiabogatenkova/miniconda3/envs/doc-py37/lib/python3.7/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
